# 3. Time-to-event Models

## Lecture errata:
There is a typo that says:

$$
\text{Bayesian inference about}  \longrightarrow L(\theta \mid y_1, \ldots, y_n) + \pi(\theta)
$$

Which should instead be:

 $$
 \text{Bayesian inference about} \longrightarrow L(\theta \mid y_1, \ldots, y_n) \cdot \pi(\theta)
 $$

## Survival models

Survival analysis studies the time $T$ until an event occurs ($T \ge 0$). Key functions:

- Cumulative Distribution Function (CDF): $F(t) = P(T \le t)$, probability event occurs by time $t$.
- Survival Function: $S(t) = P(T > t) = 1 - F(t)$, probability event has not occurred by time $t$.
- Probability Density Function (PDF): $f(t) = \frac{d}{dt}F(t) = -\frac{d}{dt}S(t)$, instantaneous rate of event at time $t$.
- Hazard Function (Hazard Rate): $h(t) = \lim_{\Delta t \to 0} \frac{P(t \le T < t+\Delta t | T \ge t)}{\Delta t}$. Instantaneous potential for event at $t$, given survival up to $t$.

$$
h(t) = \frac{f(t)}{S(t)} = -\frac{d}{dt} \ln(S(t)) 
$$

- Cumulative Hazard Function: $H(t) = \int_0^t h(u) du$. Related to survival by:

$$ 
S(t) = e^{-H(t)} = e^{-\int_0^t h(u) du} 
$$

Probability of event between $a$ and $b$ ($a<b$) is $P(a < T \le b) = F(b) - F(a) = S(a) - S(b)$.

### Exponential Distribution

Has a constant hazard rate $\lambda$.

- Parameter: Rate $\lambda > 0$.
- PDF: $f(t|\lambda) = \lambda e^{-\lambda t}$, for $t \ge 0$.
- CDF: $F(t|\lambda) = 1 - e^{-\lambda t}$.
- Survival Function: $S(t|\lambda) = e^{-\lambda t}$.
- Hazard Function: $h(t|\lambda) = \frac{\lambda e^{-\lambda t}}{e^{-\lambda t}} = \lambda$.
- Cumulative Hazard Function: $H(t|\lambda) = \lambda t$.

### Weibull Distribution

Allows hazard rates to increase ($\alpha > 1$), decrease ($\alpha < 1$), or be constant ($\alpha = 1$, Exponential case). Two common parameterizations exist.

#### Rate model (BUGS)

- Parameters: Shape $r > 0$, Rate $\lambda > 0$.
- PDF: $f(t|r, \lambda) = \lambda r t^{r-1} e^{-\lambda t^r}$, for $t > 0$.
- CDF: $F(t|r, \lambda) = 1 - e^{-\lambda t^r}$.
- Survival Function: $S(t|r, \lambda) = e^{-\lambda t^r}$.
- Hazard Function: $h(t|r, \lambda) = \frac{\lambda r t^{r-1} e^{-\lambda t^r}}{e^{-\lambda t^r}} = \lambda r t^{r-1}$.
- Cumulative Hazard Function: $H(t|r, \lambda) = \lambda t^r$.

#### Scale model (PyMC)

- Parameters: Shape $\alpha > 0$, Scale $\beta > 0$.
- PDF: $f(t|\alpha, \beta) = \frac{\alpha}{\beta} \left(\frac{t}{\beta}\right)^{\alpha - 1} e^{-(t/\beta)^{\alpha}} = \frac{\alpha}{\beta^\alpha} t^{\alpha - 1} e^{-(t/\beta)^{\alpha}}$, for $t > 0$.
- CDF: $F(t|\alpha, \beta) = 1 - e^{-(t/\beta)^\alpha}$.
- Survival Function: $S(t|\alpha, \beta) = e^{-(t/\beta)^\alpha}$.
- Hazard Function: $h(t|\alpha, \beta) = \frac{\frac{\alpha}{\beta^\alpha} t^{\alpha - 1} e^{-(t/\beta)^{\alpha}}}{e^{-(t/\beta)^\alpha}} = \frac{\alpha}{\beta^\alpha} t^{\alpha - 1} = \frac{\alpha}{\beta} \left(\frac{t}{\beta}\right)^{\alpha-1}$.
- Cumulative Hazard Function: $H(t|\alpha, \beta) = (t/\beta)^\alpha$.

#### Converting from BUGS to PyMC parameterization

Let $X$ be Weibull distributed.

BUGS parameterization: shape $r > 0$, rate $\lambda > 0$.

$$ 
f_{BUGS}(x|r, \lambda) = \lambda r x^{r-1} e^{-\lambda x^r}, \quad \text{for } x > 0 
$$

PyMC parameterization: shape $\alpha > 0$, scale $\beta > 0$.

$$ 
f_{PyMC}(x|\alpha, \beta) = \frac{\alpha}{\beta^\alpha} x^{\alpha - 1} e^{-(x/\beta)^{\alpha}}, \quad \text{for } x > 0 
$$

Relationship:
1.  $\alpha = r$
2.  $\beta = \lambda^{-1/r} = \lambda^{-1/\alpha}$

From (2), $\lambda = 1/\beta^\alpha$.

Substituting BUGS parameters with PyMC equivalents:

$$
\begin{align*}
f_{BUGS}(x|r, \lambda) &= \lambda r x^{r-1} e^{-\lambda x^r} \\
&= \left(\frac{1}{\beta^\alpha}\right) \alpha x^{\alpha-1} e^{-\left(\frac{1}{\beta^\alpha}\right) x^\alpha} \\
&= \frac{\alpha}{\beta^\alpha} x^{\alpha-1} e^{-\frac{x^\alpha}{\beta^\alpha}} \\
&= \frac{\alpha}{\beta^\alpha} x^{\alpha-1} e^{-\left(\frac{x}{\beta}\right)^\alpha} \\
&= f_{PyMC}(x|\alpha, \beta)
\end{align*}
$$

The parameterizations are equivalent if $\alpha = r$ and $\beta = \lambda^{-1/\alpha}$.

## Censoring and Truncation

Event time $T$ may not be fully observed.

Right-Censoring: Most common. Study ends or subject lost before event. True event time $T_i >$ observed censoring time $C_i$. Data is $(y_i, \delta_i)$, where $y_i = \min(T_i, C_i)$ and $\delta_i$ indicates censoring:
- $\delta_i = 0$ if event observed ($T_i \le C_i$).
- $\delta_i = 1$ if censored ($T_i > C_i$).

Left-Censoring: Event occurred before time $C_i$, exact time unknown ($T_i < C_i$).

Truncation: Differs from censoring. Subjects included only if event time is within a range (e.g., left-truncation: only subjects with $T_i > T_{start}$ are observed).

Likelihood with Right-Censoring:

For $n$ subjects ($k$ observed events, $n-k$ censored), likelihood $L$ for parameters $\theta$:
- Observed event $y_i$ ($\delta_i=0$): contribution is PDF $f(y_i|\theta)$.
- Censored time $y_i$ ($\delta_i=1$): contribution is Survival $S(y_i|\theta) = P(T_i > y_i | \theta)$.

Total likelihood:

$$ 
L(\theta | \mathbf{y}, \boldsymbol{\delta}) = \prod_{i=1}^n [f(y_i|\theta)]^{1-\delta_i} [S(y_i|\theta)]^{\delta_i} 
$$

Used for inference (MLE, Bayesian).

Types of Censoring:
- Type I Censoring: Study ends at fixed time $C$. Number of events is random.
- Type II Censoring: Study ends after fixed number of events $k$. Time $C$ is random.

